我们使用pycaret.nlp模块进行自然语言处理。

学习内容：

    设置环境：在PyCaret中设置环境并执行关键的文本预处理任务。
    创建模型：创建topic model。
    分配模型：使用经过训练的模型将文档/文本分配给主题。
    绘制图表：使用图表分析topic model/语料库。

# 1.数据集处理
数据集包含6818个样本。特征的描述如下：

    country	：借款国
    en：申请贷款时借款人的个人经历
    gender：性别（M =男性，F =女性）
    loan_amount：已批准和已支付的贷款金额
    nonpayment：贷方类型（lender	：Kiva网站上的个人注册用户，partner：与Kiva合作寻找并资助贷款的小额信贷机构）
    sector：借款人行业
    status：贷款状态（1：未还款，0：还款）

In [1]:
from pycaret.datasets import get_data
import pandas as pd


data = pd.read_csv('./data/LDA_data.csv')
data.head()

,country,en,gender,loan_amount,nonpayment,sector,status
0,Dominican Republic,"""Banco Esperanza"" is a group of 10 women looki...",F,1225,partner,Retail,0
1,Dominican Republic,"""Caminemos Hacia Adelante"" or ""Walking Forward...",F,1975,lender,Clothing,0
2,Dominican Republic,"""Creciendo Por La Union"" is a group of 10 peop...",F,2175,partner,Clothing,0
3,Dominican Republic,"""Cristo Vive"" (""Christ lives"" is a group of 10...",F,1425,partner,Clothing,0
4,Dominican Republic,"""Cristo Vive"" is a large group of 35 people, 2...",F,4025,partner,Food,0


数据集共有6818条，若电脑无法运行这么大的话，取其中1000条运行。（后续绘图会比较卡）

In [2]:
#check the shape of data
data.shape

(6818, 7)

卡就运行，不卡就别运行了。

In [3]:
# sampling the data to select only 1000 documents
data = data.sample(1000, random_state=786).reset_index(drop=True)
data.shape

(1000, 7)

# 2. 设置环境
setup()函数会初始化pycaret中的环境，并执行一些必须处理NLP问题的文本预处理步骤。在pycaret中执行任何其他功能之前，必须先调用setup()函数。

执行设置后，将自动应用以下预处理步骤：

    - 删除数字字符：从文本中删除所有数字字符。
    - 删除特殊字符：从文本中删除所有非字母数字的特殊字符。
    - Word Tokenization：单词标记化是将大量文本样本拆分为多个单词的过程。
    - 停用词的删除：很常见，即使对于语言有意义，它也对信息检索几乎没有价值。“the”，“a”，“an”，“in”
    还有很多。。。。。。
    

安装需要的语言模型
python命令行界面，例如Anaconda Prompt，在命令行中输入以下内容：
- python -m spacy download en_core_web_sm
- python -m textblob.download_corpora

然后运行下面的代码

In [6]:
from pycaret.nlp import *
exp_nlp101 = setup(data = data, target = 'en', session_id = 123)

C:\Users\liuli\anaconda3\envs\python38\lib\site-packages\mlflow\protos\scalapb\scalapb_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\liuli\anaconda3\envs\python38\lib\site-packages\mlflow\protos\scalapb\scalapb_pb2.py:29: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  options = _descriptor.FieldDescriptor(
C:\Users\liuli\anaconda3\envs\python38\lib\site-packages\mlflow\protos\scalapb\scalapb_pb2.py:37: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descrip

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\liuli\anaconda3\envs\python38\lib\site-packages\pycaret\nlp.py", line 313, in setup
    sp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  File "C:\Users\liuli\anaconda3\envs\python38\lib\site-packages\spacy\__init__.py", line 30, in load
    return util.load_model(name, **overrides)
  File "C:\Users\liuli\anaconda3\envs\python38\lib\site-packages\spacy\util.py", line 175, in load_model
    raise IOError(Errors.E050.format(name=name))
OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\liuli\anaconda3\envs\python38\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\liuli\AppData\Local\Temp/ipykernel_14328/1258638622.py

TypeError: object of type 'NoneType' has no len()

当执行setup()时，所有文本预处理步骤都会自动执行,这些步骤对于在NLP中都是必不可少的。这里极大地简化了这个过程。

# 3.创建模型
在PyCaret中创建topic model非常简单，使用create_model()函数创建topic model.该函数采用一个强制性参数，即模型名称作为字符串。此函数返回一个训练好的模型对象。

PyCaret中有5种topic model。这里我们使用LDA模型：

In [7]:
lda = create_model('lda')

INFO:logs:Initializing create_model()
INFO:logs:create_model(model=lda, multi_core=False, num_topics=None, verbose=True, system=True)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


NameError: name 'html_param' is not defined

In [8]:
print(lda)

NameError: name 'lda' is not defined

# 4. 分配模型
现在我们已经创建了一个topic model，我们将给数据集（6818个文档/样本）倒入模型加以分析结果。 我们将使用assign_model()函数来实现。

In [ ]:
lda_results = assign_model(lda)
lda_results.head()

现在dataframe里增加了6个列。Topic_0 ... Topic_3是主题比例（默认分为4类，可以修改），代表每个文档的主题分布。Dominant_Topic是具有最高比例的topic编号，而Perc_Dominant_Topic是占主导topic的比例。

# 4. 绘制图表
plot_model()函数可用于分析整个语料库或仅分析通过主题模型提取的特定主题。 

### 整个语料库的单词频率分布

In [ ]:
plot_model()

### 9.3 Topic 1的频率分布
plot_model()也可以用于分析特定topic。

In [ ]:
plot_model(lda, plot = 'frequency', topic_num = 'Topic 1')

### topic的分布情况

In [ ]:
plot_model(lda, plot = 'topic_distribution')

### t-SNE
T分布随机邻居嵌入（t-SNE）是一种非线性降维技术，非常适合在二维或三维低维空间中嵌入高维数据以进行可视化。

In [ ]:
plot_model(lda, plot = 'tsne')